In [13]:
import pandas
import csv
import matplotlib.pyplot as plt
import learn
import util
import numpy as np
import numpy.linalg as la
import scipy as sp
import scipy.spatial.distance as dist
import sys
sys.path.append ('/Users/Zhonghou/Desktop/General/Preston Lab/tesser_successor')

from tesser import network

nodes = network.node_info()
adjacency = network.adjacency (nodes)
transition = adjacency / 6


# makes the envstep necessary for learn.py
def make_envstep (directory):
    envstep = []
    objects = util.get_objects (directory)
    for index in range (len (objects)):
        obj = objects.iat[index,0]
        try:
            envstep.append (int(obj))
        except:
            pass
    return envstep

def copy (list0):
    list1 = []
    for index in range (len(list0)):
        list1.append (list0[index])
    return list1

def explore_runs (option, alpha, gamma, obj_sequence_directory):
    
    SR_matrices = []
    
    # This loop adds the address, part number and run number to the runs array, so that the object
    #     sequence in each run can be inputted to the learning agent.
    runs = []
    directory = obj_sequence_directory
    for part_num in [1]:
        for run_num in [1,2,3,4,5]:
            filename = 'tesserScan_100_B_StructLearn_Part%s_Run_%s.txt'%(part_num, run_num)
            runs.append ([directory+filename, part_num, run_num])
    
    # This option allows the SR matrix to persist across all runs from Part 1 and Part 2
    #     without ever resetting.
    if option == 'persist':
        M = np.zeros([21,21])
        for run in runs:
            part_num, run_num = run[1], run[2]
            envstep = make_envstep (run[0])
            M = np.array(learn.run_experiment(envstep, gamma, alpha, copy(M)))
            SR_matrices.append (M)
            
    if option == 'repeat':
        M = np.zeros([21,21])
        for time in range (100):
            for run in runs:
                part_num, run_num = run[1], run[2]
                envstep = make_envstep (run[0])
                M = np.array (learn.run_experiment (envstep, gamma, alpha, copy(M)))
        return M
    
    if option == 'once':
        M = np.zeros([21,21])
        for run in runs:
            part_num, run_num = run[1], run[2]
            envstep = make_envstep (run[0])
            M = np.array (learn.run_experiment (envstep, gamma, alpha, copy(M)))
        return M
    
    # This option allows the SR matrix to persist in Part 1 and Part 2, but resets it between them.
    if option == 'reset':
        M = np.zeros ([21,21])
        is_reset = False
        for run in runs:
            part_num, run_num = run[1], run[2]
            if not is_reset and part_num == 2:
                M = np.zeros ([21,21])
                is_reset = True
            envstep = make_envstep (run[0])
            M = np.array(learn.run_experiment(envstep, gamma, alpha, copy(M)))
            SR_matrices.append (M)
    
    # This option resets the SR matrix between each run.
    if option == 'independent':
        for run in runs:
            part_num, run_num = run[1], run[2]
            M = np.zeros([21,21])
            envstep = make_envstep (run[0])
            M = learn.run_experiment(envstep, gamma, alpha, M)
            SR_matrices.append (M)

    # This option forces the SR matrix to persist across all runs, but instead of plotting the SR matrix
    #     after each run, it plots the changes made to it after learning each object sequence.
    if option == 'track changes':
        M = np.zeros([21,21])
        for run in runs:
            part_num, run_num = run[1], run[2]
            envstep = make_envstep (run[0])
            M_new = np.copy(M)
            M_new = learn.run_experiment(envstep, gamma, alpha, M_new)
            SR_matrices.append (M)
            M = M_new
            
    return SR_matrices

# Modify the option in the following call to the main function in order to visualize the desired learning
#     sequence.
# explore_runs('track changes')

# df.loc[[4,17],'connect'] = 1
# df.loc[[3,11],'connect'] = 2
# df.loc[[10,18],'connect'] = 3

class Node:
    comm = {1:[1,2,3,18,19,20,21],
            2:[4,5,6,7,8,9,10],
            3:[11,12,13,14,15,16,17]}
    
    def __init__ (self, index):
        self.index = index
        self.connections = []
    
    def update_connections (self):
        if self.index == 4:
            self.connections = [5,6,7,8,9,17]
        elif self.index == 17:
            self.connections = [12,13,14,15,16,4]
        elif self.index == 3:
            self.connections = [1,2,19,20,21,11]
        elif self.index == 11:
            self.connections = [12,13,14,15,16,3]
        elif self.index == 10:
            self.connections = [5,6,7,8,9,18]
        elif self.index == 18:
            self.connections = [1,2,19,20,21,10]
        else:
            for i in range (3):
                if self.index in Node.comm[i+1]:
                    self.connections = copy(Node.comm[i+1])
                    self.connections.remove (self.index)
            

def compute_limit_matrix (gamma):
    num_states = 21
    identity = np.eye (num_states)
    return np.linalg.inv (identity - gamma*adjacency/6)

def rda (matrix):
    return dist.squareform (dist.pdist (matrix, 'correlation'))

def correlate_rows (matrix):
    return np.dot(matrix, matrix.T) / (la.norm (matrix)**2)

def correlate_columns (matrix):
    return np.dot(matrix.T, matrix) / (la.norm (matrix)**2)

def pBGivenA (A, B, C, SR):
    return SR[A][B] / (SR[A][B] + SR[A][C])

def pCGivenA (A, B, C, SR):
    return 1 - pBGivenA (A, B, C, SR)

def likelihood (cue, opt1, opt2, response, SR):
    if response == 0:
        return pBGivenA (cue, opt1, opt2, SR)
    if response == 1:
        return pCGivenA (cue, opt1, opt2, SR)

def compute_correlations(option, alpha):
    L = compute_limit_matrix (.5)
    L_vector = L.flatten()
    M = explore_runs ('repeat')
    M_vector = M.flatten()
    plt.matshow (M, vmin = 0, vmax = .5)
    plt.colorbar()
    
    if option == 'norm':
        print ('Norm of L - M: ')
        print (la.norm(L_vector - M_vector, np.inf))
    
    if option == 'correlation':
        print ('Correlation of L, M: ')
        print (np.dot (L_vector, M_vector) / (la.norm (L_vector) * la.norm (M_vector)))
        
    plt.matshow (L, vmin = 0, vmax = .5)
    plt.colorbar()
    
# This function gives the probability of obtaining the choices in the run, given
#     specific values for alpha, gamma.
    
# For obj_sequence_directory, enter the folder where the runs, which give the object sequence,
#     are stored. Use tesser scan 100B. For induction_directory, enter the full path of the 
#     induction data.

def get_log_likelihood(alpha, gamma, obj_sequence_directory, induction_directory):
    SR = explore_runs ('once', alpha, gamma, obj_sequence_directory)
    directory = induction_directory
    cue_sequence, opt1_sequence, opt2_sequence, response_sequence = util.get_induction_data (directory)
    num_trials = len (cue_sequence)
    log_likelihood = 0
    for trial_num in range (num_trials):
        try:
            cue_num, opt1_num, opt2_num, response_num = int(cue_sequence[trial_num]) - 1, int(opt1_sequence[trial_num]) - 1, int(opt2_sequence[trial_num]) - 1, int(response_sequence[trial_num]) - 1
            trial_probability = likelihood (cue_num, opt1_num, opt2_num, response_num, SR)
            log_likelihood += np.log (trial_probability)
        except ValueError:
            pass
    
    return log_likelihood

# This function finds alpha, gamma that maximize the likelihood function given by 
#     get_log_likelihood.

# For obj_sequence_directory, enter the folder where the runs, which give the object sequence,
#     are stored. Use tesser scan 100B. For induction_directory, enter the full path of the 
#     induction data.

def maximize_likelihood (obj_sequence_directory, induction_directory):
    h = 10e-3
    alpha_max, gamma_max = 0.0, 0.0
    alpha, gamma = h, h
    log_likelihood_max = get_log_likelihood (alpha, gamma, obj_sequence_directory, induction_directory)
    log_likelihoods = []
    
    
    while alpha <= 1:
        gamma = h
        while gamma <= 1:
            log_likelihood = get_log_likelihood(alpha, gamma, obj_sequence_directory, induction_directory)
            log_likelihoods.append (log_likelihood)
            if log_likelihood > log_likelihood_max:
                log_likelihood_max = log_likelihood
                alpha_max, gamma_max = alpha, gamma
            gamma += h
        alpha += h
    
    return alpha_max, gamma_max
    
# Format for the two functions below: 
# get_log_likelihood(alpha, gamma, obj_sequence_directory, induction_directory)
# maximize_likelihood(obj_sequence_directory, induction_directory)

# For obj_sequence_directory, enter the folder where the runs, which give the object sequence,
#     are stored. Use tesser scan 100B. For induction_directory, enter the full path of the 
#     induction data.

obj_sequence_directory = '/Users/Zhonghou/Desktop/General/Preston Lab/tesser_successor/tesserScan_100_B/'
induction_directory = '/Users/Zhonghou/Desktop/General/Preston Lab/tesser_successor/tesserScan_100_B/tesserScan_100_B_InductGen.txt'
alpha, gamma = .6, 1

print ('Log likelihood:', get_log_likelihood (alpha, gamma, obj_sequence_directory, induction_directory))
print ('Max alpha, gamma:', maximize_likelihood(obj_sequence_directory, induction_directory))

Log likelihood: -48.77909548935698


/Users/Zhonghou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:213: RuntimeWarning: divide by zero encountered in log


Max alpha, gamma: (0.5500000000000003, 0.9900000000000007)
